In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
class InvalidMove(Exception):
    pass

In [8]:
raise InvalidMove('asdf')

InvalidMove: asdf

In [106]:
class Board:
    def __init__(self,size = (10,10),markers = ['X','O']):
        self.size = size
        self.board = np.zeros(size)
        self.active_player = 0
        self.set_markers(markers)
        self.__moves = {0:[],1:[]}
        self.scores = [0,0]
        self.strikes = {0:[],1:[]}
        
        
    def next_move(self,move):
        if self.board[move]>0:
            raise InvalidMove(f'The selected cell [{move}] is already full.')
        points = self.__move(self.active_player,move)
        self.next_player()
        
    def __move(self,player,move):
        self.board[move] = player+1
        self.__moves[player].append(move)
        points = self.check_for_points(move,player)
        self.scores[player] += points
        return points
        
    def check_for_points(self,move,player):
        points = 0
        piece = self.__get_piece(move)
        potential_points = []
        for i in range(4):
            #check rows
            row = piece[3,i:i+4]
            if self.__is_strike(row,player):
                cells = {(move[0],move[1]+i-3+c) for c in range(4)}
                potential_points.append(cells)
            # check columns
            row = piece[i:i+4,3]
            if self.__is_strike(row,player):
                cells = {(move[0]+i-3+c,move[1]) for c in range(4)}
                potential_points.append(cells)
            # check main diagonal
            row = np.array([piece[i+c,i+c] for c in range(4)])
            if self.__is_strike(row,player):
                cells = {(move[0]+i-3+c,move[1]+i-3+c) for c in range(4)}
                potential_points.append(cells)
            # check secondary diagonal
            row = np.array([piece[i+c,6-i-c] for c in range(4)])
            if self.__is_strike(row,player):
                cells = {(move[0]+i-3+c,move[1]+6-(i-3+c)) for c in range(4)}
                potential_points.append(cells)
        
        for cells in potential_points:
            possible = True
            for strike in self.strikes:
                if len(cells.intersection(strike))>1:
                    possible = False
                    break
            if possible:
                points += 1
                self.strikes.append(cells)
        return points
    
    def __is_strike(self,cells,player):
        cell_set = set(cells)
        if len(cell_set)==1 and player in cell_set:
            return True
        else:
            return False
        
    def __get_piece(self,move):
        size = self.size
        temp = np.zeros((size[0]+6,size[1]+6))
        temp[3:-3,3:-3] = self.board
        i1 = move[0]
        i2 = move[0]+7
        j1 = move[1]
        j2 = move[1]+7
        print(temp[i1:i2,j1:j2])
        return temp[i1:i2,j1:j2]
        
    def get_moves(self):
        return self.__moves
    
    def next_player(self):
        self.active_player = 1- self.active_player
        
    def set_markers(self,markers):
        if isinstance(markers,list) and len(markers)==2:
            self.markers = markers
            
    def render(self):
        print('Player A:',self.scores[0])
        print('Player B:',self.scores[1])
        view = np.empty(self.size,str)
        view[:,:] = ' '
        view[self.board==1] = self.markers[0]
        view[self.board==2] = self.markers[1]
        print(view)
        
    def test(self):
        self.__get_piece((2,2))

In [110]:
brd = Board()

In [111]:
brd.render()

Player A: 0
Player B: 0
[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']]


In [112]:

for i in range(5):
    brd.next_move((1,i))
    brd.next_move((2,i))
brd.render()

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 2. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0.]
 [0. 2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0.]
 [0. 2. 2. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 

In [79]:
brd.test()

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [60]:
a = [1,2,3]

In [82]:
a=set([1,1,1])

In [83]:
1 in a

True